# Importar Librerías Necesarias
Importar las librerías requeridas como numpy, pandas, tensorflow, matplotlib, seaborn, entre otras.

In [ ]:
!pip install numpy pandas tensorflow keras keras-tuner scikit-learn matplotlib seaborn

In [ ]:
# Importar las librerías necesarias
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner.tuners import Hyperband
from keras_tuner.engine.hyperparameters import HyperParameters
from keras import callbacks
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'pandas'

# Cargar y Preprocesar los Datos
Cargar el archivo CSV, mezclar los datos y separar las características (X) y los objetivos (Y).

In [ ]:
# Cargar y Preprocesar los Datos

# Cargar los datos desde el archivo CSV
dataset = pd.read_csv('original_noise10_Train_set')

# Reiniciar los índices del DataFrame
dataset = dataset.reset_index(drop=True)

# Mezclar los datos aleatoriamente
dataset = dataset.sample(frac=1).reset_index(drop=True)

# Separar las características (X) y los objetivos (Y)
X = dataset.iloc[:, 3:-2].values  # Seleccionar columnas de características
Y = dataset.iloc[:, -2:].values   # Seleccionar columnas de objetivos

# Definir el Modelo de Red Neuronal
Utilizar la clase Sequential de Keras para definir una red neuronal con capas densas.

In [ ]:
# Definir el Modelo de Red Neuronal
model = Sequential()

# Agregar la primera capa densa con 64 neuronas y función de activación ReLU
model.add(Dense(64, input_dim=X.shape[1], activation='relu'))

# Agregar una segunda capa densa con 32 neuronas y función de activación ReLU
model.add(Dense(32, activation='relu'))

# Agregar la capa de salida con 2 neuronas (una por cada objetivo) y función de activación lineal
model.add(Dense(Y.shape[1], activation='linear'))

# Compilar el modelo con el optimizador Adam y la métrica de error cuadrático medio
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Mostrar un resumen del modelo
model.summary()

# Compilar y Entrenar el Modelo
Configurar el optimizador, la función de pérdida y las métricas. Entrenar el modelo utilizando los datos preprocesados.

In [ ]:
# Compilar y Entrenar el Modelo

# Configurar EarlyStopping para detener el entrenamiento si no hay mejora en el conjunto de validación
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Dividir los datos en conjuntos de entrenamiento y validación
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# Entrenar el modelo con los datos de entrenamiento y validación
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

# Graficar la pérdida y la métrica de entrenamiento y validación
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Pérdida de Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de Validación')
plt.title('Pérdida durante el Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(history.history['mae'], label='MAE de Entrenamiento')
plt.plot(history.history['val_mae'], label='MAE de Validación')
plt.title('MAE durante el Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('MAE')
plt.legend()
plt.show()

# Evaluar el Modelo
Evaluar el rendimiento del modelo utilizando métricas relevantes y validación cruzada.

In [ ]:
# Evaluar el Modelo

# Configurar validación cruzada con 5 particiones
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Listas para almacenar las métricas de cada partición
mse_scores = []
mae_scores = []

# Realizar validación cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    # Entrenar el modelo en los datos de entrenamiento
    model.fit(X_train, Y_train, epochs=50, batch_size=32, verbose=0)
    
    # Evaluar el modelo en los datos de prueba
    scores = model.evaluate(X_test, Y_test, verbose=0)
    mse_scores.append(scores[0])  # Pérdida (MSE)
    mae_scores.append(scores[1])  # MAE

# Calcular las métricas promedio
mean_mse = np.mean(mse_scores)
mean_mae = np.mean(mae_scores)

# Mostrar los resultados
print(f"MSE promedio en validación cruzada: {mean_mse:.4f}")
print(f"MAE promedio en validación cruzada: {mean_mae:.4f}")

# Graficar las métricas de validación cruzada
plt.figure(figsize=(12, 6))
plt.bar(range(1, 6), mse_scores, label='MSE por partición', color='skyblue')
plt.axhline(mean_mse, color='red', linestyle='--', label='MSE Promedio')
plt.title('MSE en Validación Cruzada')
plt.xlabel('Partición')
plt.ylabel('MSE')
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
plt.bar(range(1, 6), mae_scores, label='MAE por partición', color='lightgreen')
plt.axhline(mean_mae, color='red', linestyle='--', label='MAE Promedio')
plt.title('MAE en Validación Cruzada')
plt.xlabel('Partición')
plt.ylabel('MAE')
plt.legend()
plt.show()

# Visualizar Resultados
Generar gráficos para visualizar el rendimiento del modelo, como la pérdida y la precisión durante el entrenamiento.

In [ ]:
# Visualizar Resultados

# Graficar la pérdida y la métrica de entrenamiento y validación
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Pérdida de Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de Validación')
plt.title('Pérdida durante el Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(history.history['mae'], label='MAE de Entrenamiento')
plt.plot(history.history['val_mae'], label='MAE de Validación')
plt.title('MAE durante el Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('MAE')
plt.legend()
plt.show()

# Graficar las métricas de validación cruzada
plt.figure(figsize=(12, 6))
plt.bar(range(1, 6), mse_scores, label='MSE por partición', color='skyblue')
plt.axhline(mean_mse, color='red', linestyle='--', label='MSE Promedio')
plt.title('MSE en Validación Cruzada')
plt.xlabel('Partición')
plt.ylabel('MSE')
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
plt.bar(range(1, 6), mae_scores, label='MAE por partición', color='lightgreen')
plt.axhline(mean_mae, color='red', linestyle='--', label='MAE Promedio')
plt.title('MAE en Validación Cruzada')
plt.xlabel('Partición')
plt.ylabel('MAE')
plt.legend()
plt.show()

: 